# 理论基础
详细可见《统计学习方法第二版》第七章
## SVM的特点
- 优点：
    - 泛化错误率低
    - 计算开销不大
    - 结果易解释
- 缺点：
    - 对参数调节和核函数选择敏感
    - 原始分类器不加修改仅适用于处理二分类问题
- 使用数据类型：
    - 数值型
    - 标称型
---
## 背景
- 对于线性可分数据，存在多个N-1维分割超平面来对数据进行划分（也称为分类决策边界）
- 距离分割超平面最近的点确保其尽可能远，这里点到分割面的距离称为**间隔**，直觉上我们希望间隔尽可能大
- **支持向量**就是距离分割超平面最近的那些点，我们要尝试最大化支持向量与分割面的距离，需要找此问题的优化求解方法

---

## 寻找最大间隔
- N维空间上点到N-1维超平面的距离为$\frac{|W^TX+b|}{||W||}$
- 这里类标签使用{-1,+1}实际上和{0,1}等价，但是数学上会方便处理
- 我们需要找到最小间隔的数据点，也就是支持向量。随后我们需要对间隔最大化
$$
 arg\max_{w,b}\{{\min_n(y \cdot \frac{W^TX+b}{||W||})}\}\\
 s.t. \quad y\cdot(W^TX+b)\geq 1.0
$$
- 对于带约束的优化问题，可以使用拉格朗日乘子法，最终优化目标函数可以简化成：
$$
\max_\alpha \{  \sum^m_{i=1}\alpha_i -\frac12\sum^m_{i,j=1}{y^iy^j\alpha_i \alpha_j<x^i,x^j>} \}\\
s.t. \quad \alpha \geq 0,\sum^m_{i=1}\alpha_i\cdot y^i = 0\\
f(x) = W^Tx+b = \sum^m_{i=1}{\alpha_i y_i x_i^Tx} + b
$$
- 至此我们有个数据必须100%线性可分，然而由于并非所有数据线性可分我们可以引入**松弛变量**，适当允许部分数据点分类错误，此时需要更新约束条件为$C\geq \alpha \geq 0, \sum^m_{i=1}\alpha_i\cdot y^i = 0$
- 常数C用于控制*最大化间隔*和*保证大部分点的函数间隔小于1.0*这两个目标的权重
- 一旦求出了$\alpha$分割超平面可以使用alpha表示，这一结论十分直接，SVM就是求解这些$\alpha$
---

## SVM的一般框架
1. 收集数据
2. 准备数据：数值型数据
3. 分析数据：有助于可视化分隔超平面
4. 训练算法：实现对两个参数的调优
5. 测试算法
6. 使用算法：几乎所有分类问题都可以使用SVM，值得一提的是SVM被本身是二分类器，对多分类应用需要做出一些修改

# SMO（Sequential Minimal Optimization）高效算法
- 在此，我们需要对最后两个式子进行优化，其中一个是最小化的目标函数，另个一是优化过程中需要遵循的约束
- 早期，人们使用二次规划求解工具求解上述最优化问题，这种工具是用于在线约束下优化多个变量的二次目标函数的软件，其通过强大的算力支撑，实现也十分复杂
- 一旦alpha的最优质的确定，我们就得到了分割超平面并能将之用于数据分类
---
- 1996年，John Platt发布了一个称为SMO的强大算法，用于训练SVM，其将大优化问题分解成多个小优化问题求解，小优化问题容易求解，对他们顺序求解与整体求解结果完全一致，且SMO求解时间短很多
- 原理
> 每次循环选两个alpha进行优化，一旦找到一对合适的alpha，增加其中一个同时减小另一个。这里的合适是两个alpha必须满足一定条件
>- 条件之一就是这两个alpha必须要在间隔边界之外
>- 第二个条件则是两个alpha还没有进行过区间化处理或者不在边界上
---

## 简化版的SMO
- 首先进行简化版的SMO算法，此处虽然代码少，但执行速度慢
- 简化版跳过了确定选择最优alpha对，而是在数据集上遍历每个alpha，然后再剩下的集合中随机选一个来构建alpha对
- 之所以要同时改变两个alpha对，是因为我们存在约束$\sum\alpha_i\cdot y^i = 0$,改变一个alpha会导致约束失效，因此需要总同时改变两个
- 对此，我们构建一个辅助函数，用于在某个区间内随机选择一个整数，同时也要构造另一个辅助函数，用于在数值太大时对其调整

In [1]:
# 加载数据
def loadDataSet(fileName):
    dataMat = []
    labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]),float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat

import random
# 随机选择另一个alpha
def selectJrand(i,m):
    j = i
    while(j==i):
        j = int(random.uniform(0,m))
    return j

# 调整alpha在H和L之间
def clipAlpha(aj,H,L):
    if aj>H:
        aj = H
    if aj<L:
        aj = L
    return aj

In [2]:
dataArr,labelArr = loadDataSet('../data/SVM/testSet.txt')
print(labelArr)

[-1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]


In [3]:
import numpy as np

def smoSimple(dataMatIn,classLabels,C,toler,maxIter):
    dataMatrix = np.mat(dataMatIn)
    labelMat = np.mat(classLabels).transpose()
    b = 0
    m,n = dataMatrix.shape
    alphas = np.mat(np.zeros((m,1)))
    curIter = 0
    while (curIter<maxIter):
        # 用于检测alpha是否变化
        alphaPairsChanged = 0
        for i in range(m):
            fxi = float( np.multiply(alphas,labelMat).T * (dataMatrix*dataMatrix[i,:].T) ) + b
            Ei = fxi - float(labelMat[i])
            # 检测alpha_i是否在两个间隔之外
            if ( (labelMat[i]*Ei < -toler) and (alphas[i] < C) ) or ( (labelMat[i]*Ei > toler) and (alphas[i]>0) ):
                j = selectJrand(i,m)
                fxj = float( np.multiply(alphas,labelMat).T * (dataMatrix*dataMatrix[j,:].T) ) + b
                Ej = fxj - float(labelMat[j])

                alphaIold = alphas[i].copy()
                alphaJold = alphas[j].copy()

                if (labelMat[i] != labelMat[j]):
                    k = alphas[j] - alphas[i]
                    L = max(0,k)
                    H = min(C,C+k)
                else:
                    k = alphas[j] + alphas[i]
                    L = max(0,k-C)
                    H = min(C,k)

                if L==H:
                    # print('L==H')
                    continue
                    
                    
                eta = 2.0 * dataMatrix[i,:] * dataMatrix[j,:].T - \
                        dataMatrix[i,:]*dataMatrix[i,:].T-\
                        dataMatrix[j,:]*dataMatrix[j,:].T
                if eta >= 0:
                    # print('eta >= 0')
                    continue

                alphas[j] -=labelMat[j]*(Ei-Ej)/eta
                alphas[j] = clipAlpha(alphas[j],H,L)

                if(abs(alphas[j]-alphaJold)<0.00001):
                    # print('j not moving enough')
                    continue
                alphas[i] += labelMat[i]*labelMat[j]*(alphaJold-alphas[j])
                b1 = b-Ei-labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T -\
                     labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
                b2 = b-Ej-labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T -\
                     labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T     

                if (0<alphas[i]) and (alphas[i]<C):
                    b = b1
                elif (0<alphas[j]) and (alphas[j]<C):
                    b = b2
                else:
                    b = (b1+b2)/2.0
                alphaPairsChanged += 1
                # print('iter: %d i: %d, pairs changed %d'%(curIter,i,alphaPairsChanged))
        if (alphaPairsChanged == 0):
            curIter += 1
        else:
            curIter = 0
        # print('iteration number： %d'% curIter)     
    return b,alphas


In [4]:
import time

t0 = time.clock()
b,alphas = smoSimple(dataArr, labelArr, 0.6, 0.001, 40)
print(time.clock() - t0, "seconds process time")

4.3119043 seconds process time


In [5]:
b

matrix([[-3.77297324]])

In [6]:
alphas[alphas>0]

matrix([[0.16820858, 0.09993403, 0.09437796, 0.36252057]])

In [7]:
for i in range(100):
    if alphas[i]>0:
        print(dataArr[i],labelArr[i])

[4.658191, 3.507396] -1.0
[3.457096, -0.082216] -1.0
[2.893743, -1.643468] -1.0
[6.080573, 0.418886] 1.0
